### Data Preprossing------------------------------------

In [163]:
import os   # for interacting with the operating system,  include many functions to interact with the file system
import numpy as np  # Python library used for working with arrays
import pandas as pd  #data manipulation and analysis
import re

from datetime import date
today = date.today()

from functools import reduce


In [93]:
# list all files in directory - Raw Data
os.getcwd()

'c:\\Users\\shyam.kalliadan\\Desktop\\ESR10\\programming\\ESR_10'

### 1. Data Column Rename -------------------

In [94]:
raw_data_directory = os.listdir('Raw_data')
processed_data_directory = os.listdir('processed_data')

In [95]:
def mapper(old_name):
    x = re.sub("^(sub_DAQ.mod_LogicLev.cntr)\\_([a-zA-Z0-9]+)\\.([a-zA-Z]+)\\_", "", old_name)
    x = re.sub("\\.(a_var)$", "", x)
    x = re.sub(" and ", "_", x)
    return x

In [96]:
for raw_file in raw_data_directory:
    pbr_dir = re.split("\\_", raw_file)
    pbr_dir = pbr_dir[0]
    new_path = os.path.join('processed_data', pbr_dir)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    fpc_list = os.listdir(new_path)
    if raw_file not in fpc_list:
        raw_file_path = os.path.join('Raw_data', raw_file)
        raw_data_frame = pd.read_csv(raw_file_path, sep=';')
        raw_data = raw_data_frame.rename(columns=mapper)

        save_path = os.path.join(new_path, raw_file)
        raw_data.to_csv(save_path, index = False)
        os.remove(raw_file_path)
        

#### 2. Data Merging and column drop

In [152]:
processed_dir = './processed_data'

species_dir_list = [dir for dir in os.listdir(processed_dir)]
print(species_dir_list)

['FPC11-Rhodomonas-salina', 'FPC12-Diacronema-lutheri']


In [232]:
def merger_function(dir,species_dir):
    
    combined_df = pd.DataFrame()
    print(combined_df)
    temp_filename = re.split("\\\\", species_dir)
    today_date = today.strftime("%b-%d-%Y")
    combined_file_name = temp_filename[1] + '_' + today_date

    for sensor_file in dir:
        processed_file_path = os.path.join(species_dir, sensor_file)
        print(processed_file_path)
        df_temp_data = pd.read_csv(processed_file_path)
        df_temp_data = df_temp_data.drop(['us'], axis = 1)

        if combined_df.empty:
            combined_df = df_temp_data
        else:
            combined_df=pd.merge(combined_df, df_temp_data, on='Date_Time', how='inner')

    print('----------------------------------')
    combined_df = combined_df.drop(['C_PAR_x', 'C_PAR_y'], axis = 1)
    print(combined_df.info())




    

    

In [233]:
for dir in species_dir_list:
    species_dir = os.path.join(processed_dir,dir)
    dir = list()
    for species_file in os.listdir(species_dir):
        dir.append(species_file)
    
    merger_function(dir,species_dir)


Empty DataFrame
Columns: []
Index: []
./processed_data\FPC11-Rhodomonas-salina\FPC11-Rhodomonas-salina_12_12_21_to_31_01_22_light_temp_ph.csv
./processed_data\FPC11-Rhodomonas-salina\FPC11-Rhodomonas-salina_12_12_21_to_31_01_22_productivity_efficiencycsv.csv
./processed_data\FPC11-Rhodomonas-salina\FPC11-Rhodomonas-salina_13_12_21_to_31_01_22_flow_and_pressure.csv
----------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2208 entries, 0 to 2207
Data columns (total 27 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date_Time  2208 non-null   object 
 1   QT         2208 non-null   float64
 2   QC         2208 non-null   int64  
 3   QB         2208 non-null   int64  
 4   QR         2208 non-null   int64  
 5   QG         2208 non-null   int64  
 6   QT_corr    2208 non-null   float64
 7   TT0        2208 non-null   float64
 8   TT1        2208 non-null   float64
 9   pH         2208 non-null   float64
 10  LReact